<a href="https://colab.research.google.com/github/awadell1/MLOnAGooseberryBush/blob/master/src/TF_Word2Vec_no_clean_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Warning: if you have a 8G RAM laptop, don't run it locally. This code will take up approximately 10G of RAM.
#Highly recommend run it on google colab in GPU mode


In [ ]:
import io
import itertools
import numpy as np
import os
import re
import string

import tensorflow as tf

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
SEED = 42 #setting the seed for reproducebility
AUTOTUNE = tf.data.experimental.AUTOTUNE #currently not entirely sure what this can do

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from google.colab import drive
from tqdm.notebook import tqdm #create a progress bar
from keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import skipgrams
drive.mount('/content/drive')

Mounted at /content/drive


Part 1 Using Non-spell check version

In [ ]:
def sentence_printout(path):
    #just to demonstrate the txt file (the first ten sentence)
    sentence=[]
    with open(path) as f: 
        lines = f.read().splitlines()
    sentence = []
    for line in lines:
        filter(None, line.split('\t'))
        sentence.append(list(filter(None, line.split('\t'))))
    f.close()
    return sentence

In [ ]:
sentence=sentence_printout(path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/description_sentence_newest.txt')

In [ ]:
I2W_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/for_final_submission/Dictionary_new.csv')
I2W=I2W_df.to_dict(orient='dict')['Vocab']
I2W[0] = ''
W2I = {v: k for k, v in I2W.items()}

In [ ]:
def sentence_to_indices(data,w2ind):
    sentences_num=[]
    for row in tqdm(data):
        temp_mem=[w2ind[word] for word in row]
        sentences_num.append(temp_mem)
    return sentences_num

In [ ]:
sentences_num = sentence_to_indices(data=sentence,w2ind=W2I)

In [ ]:
max_sentence_length= max([len(seq) for seq in sentences_num])

In [ ]:
for seq in sentences_num[:2]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[15, 873, 202, 13, 1978, 3581, 1, 88, 105] 
 => 
 ['aromas', 'include', 'tropical', 'fruit', 'broom', 'brimstone', 'and', 'dried', 'herb']
[2, 16, 908, 1030, 928, 308, 14444, 48, 60, 1, 88, 421, 142, 360, 18] 
 => 
 ['the', 'palate', "isn't", 'overly', 'expressive', 'offering', 'unripened', 'apple', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acidity']


In [ ]:
#this function could be a bit confusing. The core idea of this function is to generate train data. 
#you can think of this as: 
#now, we have the description encoded as the above vector
#we want to generate the target word (the word we select) and context word (the word we want to predict)
#this function not only use positive skip gram but also use negative sampling
#positive skip gram: target word + correct context word 
#negative sampling: target word + some random word selected from the dictonary (garanteed not in the context)

#the label will be [1,0,0,0,0]
#the first label is the saying the context word is the correct prediction
#the number of 0 dependes on how many negative samples you want to generate


# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.

def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      target_word = tf.cast(target_word,dtype='int64')
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
#calling this function
num_ns=3 #number of negative samples for each target word
window_size=2 #how long is your window_size, a good visualization: https://www.tensorflow.org/tutorials/text/images/word2vec_negative_sampling.png
vocab_size = len(W2I)
targets, contexts, labels = generate_training_data(
    sequences=sentences_num, 
    window_size=window_size, 
    num_ns=num_ns, 
    vocab_size=vocab_size, 
    seed=SEED)

In [ ]:
print(len(targets),len(contexts),len(labels))

2395059 2395059 2395059


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset shapes: (((1024,), (1024, 4, 1)), (1024, 4)), types: ((tf.int64, tf.int64), tf.int64)>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 4, 1)), (1024, 4)), types: ((tf.int64, tf.int64), tf.int64)>


In [ ]:
#this is the Word2Vec architecture 
#a detailed description can be found here: https://www.tensorflow.org/tutorials/text/word2vec
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)

part 1-1 100D

In [ ]:
#compling the model
embedding_dim = 100
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#trained 50 epochs before the 10 epochs
word2vec.fit(dataset,epochs=10,verbose=1)

Epoch 1/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3181 - accuracy: 0.8736
Epoch 2/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3173 - accuracy: 0.8738
Epoch 3/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3166 - accuracy: 0.8741
Epoch 4/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3159 - accuracy: 0.8743
Epoch 5/10
2338/2338 [==============================] - 92s 40ms/step - loss: 0.3152 - accuracy: 0.8745
Epoch 6/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3146 - accuracy: 0.8747
Epoch 7/10
2338/2338 [==============================] - 93s 40ms/step - loss: 0.3139 - accuracy: 0.8749
Epoch 8/10
2338/2338 [==============================] - 92s 40ms/step - loss: 0.3133 - accuracy: 0.8751
Epoch 9/10
2338/2338 [==============================] - 92s 40ms/step - loss: 0.3127 - accuracy: 0.8752
Epoch 10/10
2338/2338 [==============================] - 92s 39m

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]

In [ ]:
out_v = io.open('/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors100.tsv', 'w', encoding='utf-8')
out_m = io.open('/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/metadata100.tsv', 'w', encoding='utf-8')

for index, word in I2W.items():
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

part 1-2 300D

In [ ]:
#compling the model
embedding_dim = 300
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#trained 30 steps
word2vec.fit(dataset,epochs=1,verbose=1)

2338/2338 [==============================] - 252s 108ms/step - loss: 0.2617 - accuracy: 0.8888


In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]

In [ ]:
out_v = io.open('/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors300.tsv', 'w', encoding='utf-8')
out_m = io.open('/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/metadata300.tsv', 'w', encoding='utf-8')

for index, word in I2W.items():
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()